In [1]:
import LibGEOS
import GeoJSON

In [2]:
SchoolDistrict_borders=GeoJSON.parsefile("NYC_data/processed/SchoolDistrict_borders/SchoolDistrict_borders.json")

GeoJSON.FeatureCollection(GeoJSON.Feature[GeoJSON.Feature(GeoJSON.LineString(Array{Float64,1}[[1.04771e6,2.2437e5],[1.04778e6,2.24293e5],[1.04773e6,2.24262e5],[1.04771e6,2.24242e5],[1.04769e6,2.24228e5],[1.04767e6,224216.0],[1.04744e6,224054.0],[1.0473e6,2.23944e5],[1.04704e6,2.23757e5],[1.04683e6,2.23601e5]  …  [1.03952e6,2.03953e5],[1.03943e6,2.03909e5],[1.03955e6,2.03879e5],[1.03958e6,2.03364e5],[1.03952e6,2.03301e5],[1.03964e6,2.03242e5],[1.03968e6,2.02834e5],[1.03963e6,2.02745e5],[1.03963e6,2.02726e5],[1.03954e6,2.02685e5]],#undef,#undef),Dict{String,Any}(Pair{String,Any}("SchoolDistrict2",26),Pair{String,Any}("SchoolDistrict1",25)),#undef,#undef,"0"),GeoJSON.Feature(GeoJSON.LineString(Array{Float64,1}[[1.04771e6,2.2437e5],[1.04778e6,2.24293e5],[1.04773e6,2.24262e5],[1.04771e6,2.24242e5],[1.04769e6,2.24228e5],[1.04767e6,224216.0],[1.04744e6,224054.0],[1.0473e6,2.23944e5],[1.04704e6,2.23757e5],[1.04683e6,2.23601e5]  …  [1.03952e6,2.03953e5],[1.03943e6,2.03909e5],[1.03955e6,2.03879e

In [31]:
ls_json = GeoJSON.LineString([[1.0,2.0],[3.0,4.0]])

GeoJSON.LineString(Array{Float64,1}[[1.0,2.0],[3.0,4.0]],#undef,#undef)

In [33]:
methods(LibGEOS.LineString)

# 4 methods for generic function "(::Type)":
LibGEOS.LineString(ptr::Ptr{Ptr{Void}}) at /Users/imolk/.julia/v0.5/LibGEOS/src/geos_types.jl:31
LibGEOS.LineString(line::Array{Array{Float64,1},1}) at /Users/imolk/.julia/v0.5/LibGEOS/src/geos_types.jl:35
LibGEOS.LineString{T<:GeoInterface.AbstractLineString}(obj::T) at /Users/imolk/.julia/v0.5/LibGEOS/src/geos_types.jl:36
(::Type{T}){T}(arg) at sysimg.jl:53

In [34]:
LibGEOS.LineString(ls_json.coordinates)

LibGEOS.LineString(Ptr{Ptr{Void}} @0x00007fb0d764f570)

In [30]:
GeoJSON.LineString([[1.0,2.0],[3.0,4.0]])

GeoJSON.LineString(Array{Float64,1}[[1.0,2.0],[3.0,4.0]],#undef,#undef)

In [29]:
SchoolDistrict_borders.features[1].geometry

GeoJSON.LineString(Array{Float64,1}[[1.04771e6,2.2437e5],[1.04778e6,2.24293e5],[1.04773e6,2.24262e5],[1.04771e6,2.24242e5],[1.04769e6,2.24228e5],[1.04767e6,224216.0],[1.04744e6,224054.0],[1.0473e6,2.23944e5],[1.04704e6,2.23757e5],[1.04683e6,2.23601e5]  …  [1.03952e6,2.03953e5],[1.03943e6,2.03909e5],[1.03955e6,2.03879e5],[1.03958e6,2.03364e5],[1.03952e6,2.03301e5],[1.03964e6,2.03242e5],[1.03968e6,2.02834e5],[1.03963e6,2.02745e5],[1.03963e6,2.02726e5],[1.03954e6,2.02685e5]],#undef,#undef)

In [25]:
LibGEOS.LineString(SchoolDistrict_borders.features[1].geometry)

LoadError: MethodError: Cannot `convert` an object of type GeoJSON.LineString to an object of type LibGEOS.LineString
This may have arisen from a call to the constructor LibGEOS.LineString(...),
since type constructors fall back to convert methods.

In [27]:
LibGEOS.LineString <: GeoInterface.AbstractLineString

true

In [28]:
GeoJSON.LineString <: GeoInterface.AbstractLineString

false

In [26]:
LibGEOS.LineString()

LibGEOS.LineString

In [39]:
LibGEOS.MultiLineString(obj::GeoJSON.MultiLineString) = LibGEOS.MultiLineString(GeoJSON.coordinates(obj))
LibGEOS.LineString(obj::GeoJSON.LineString) = LibGEOS.LineString(GeoJSON.coordinates(obj))

In [40]:
borders_dict = Dict{Tuple{Int,Int}, Union{LibGEOS.MultiLineString,LibGEOS.LineString}}()
convert_geos(geom::GeoJSON.LineString) = LibGEOS.LineString(geom)
convert_geos(geom::GeoJSON.MultiLineString) = LibGEOS.MultiLineString(geom)

for f in SchoolDistrict_borders.features
    schdist1 = f.properties["SchoolDistrict1"]
    schdist2 = f.properties["SchoolDistrict2"]
    border = convert_geos(f.geometry)
    borders_dict[schdist1,schdist2] = border
end

# NYC

In [41]:
using DataFrames

In [42]:
NYC_sales=readtable("NYC_data/processed/NYC_sales.csv", header=true)
begin
    nyc_schdistrs = NYC_sales[:SchDistr]
    schd_strings = [dec(sd,2) for sd in  nyc_schdistrs.values]
    str_schdistrs = NullableCategoricalArray(schd_strings, nyc_schdistrs.isnull)
    str_schdistrs
    NYC_sales[:SchDistr] = str_schdistrs
end
# categorical variables
categorical!(NYC_sales, [:BOROUGH,
    :BUILDING_CLASS_CATEGORY,
    :BUILDING_CLASS_AT_TIME_OF_SALE,
    :ZIP_CODE,
    :TAX_CLASS_AT_PRESENT,
    :TAX_CLASS_AT_TIME_OF_SALE,
    :NEIGHBORHOOD,
    ])   

In [43]:
residential = Dict(
"01  ONE FAMILY DWELLINGS"=>true,
"02  TWO FAMILY DWELLINGS"=>true,
"03  THREE FAMILY DWELLINGS"=>true,
"04  TAX CLASS 1 CONDOS"=>true,
"05  TAX CLASS 1 VACANT LAND"=>false,
"06  TAX CLASS 1 - OTHER"=>false,
"07  RENTALS - WALKUP APARTMENTS"=>false,
"08  RENTALS - ELEVATOR APARTMENTS"=>false,
"09  COOPS - WALKUP APARTMENTS"=>true,
"10  COOPS - ELEVATOR APARTMENTS"=>true,
"11  SPECIAL CONDO BILLING LOTS"=>false,
"11A CONDO-RENTALS"=>false,
"12  CONDOS - WALKUP APARTMENTS"=>true, # why are these duplicated?
"13  CONDOS - ELEVATOR APARTMENTS"=>true,
"14  RENTALS - 4-10 UNIT"=>false,
"15  CONDOS - 2-10 UNIT RESIDENTIAL"=>true,
"16  CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT"=>false,
"17  CONDO COOPS"=>true,
"18  TAX CLASS 3 - UNTILITY PROPERTIES"=>false,
"21  OFFICE BUILDINGS"=>false,
"22  STORE BUILDINGS"=>false,
"23  LOFT BUILDINGS"=>false,
"25  LUXURY HOTELS"=>false,
"26  OTHER HOTELS"=>false,
"27  FACTORIES"=>false,
"28  COMMERCIAL CONDOS"=>false,
"29  COMMERCIAL GARAGES"=>false,
"30  WAREHOUSES"=>false,
"31  COMMERCIAL VACANT LAND"=>false,
"32  HOSPITAL AND HEALTH FACILITIES"=>false,
"33  EDUCATIONAL FACILITIES"=>false,
"34  THEATRES"=>false,
"35  INDOOR PUBLIC AND CULTURAL FACILITIES"=>false,
"36  OUTDOOR RECREATIONAL FACILITIES"=>false,
"37  RELIGIOUS FACILITIES"=>false,
"38  ASYLUMS AND HOMES"=>false,
"41  TAX CLASS 4 - OTHER"=>false,
"42  CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC"=>false,
"43  CONDO OFFICE BUILDINGS"=>false,
"44  CONDO PARKING"=>false,
"45  CONDO HOTELS"=>false,
"46  CONDO STORE BUILDINGS"=>false,
"47  CONDO NON-BUSINESS STORAGE"=>false,
"48  CONDO TERRACES/GARDENS/CABANAS"=>false,
"49  CONDO WAREHOUSES/FACTORY/INDUS"=>false,
);

In [44]:
NYC_sales[:logSalePricePerSQFT] = map(log, NYC_sales[:SALE_PRICE]) .- map(log, NYC_sales[:GROSS_SQUARE_FEET])
believable = zeros(Bool, size(NYC_sales,1))
removed=zeros(Int, 12)
for i in 1:size(NYC_sales,1)
    if NYC_sales[i,:SALE_PRICE].isnull
        removed[1] += 1
        continue
    end
    if NYC_sales[i,:SchDistr].isnull
        removed[2] += 1
        continue
    end
    if NYC_sales[i,:BUILDING_CLASS_AT_TIME_OF_SALE].isnull
        removed[3] += 1
        continue
    end
    if NYC_sales[i,:BUILDING_CLASS_CATEGORY].isnull
        removed[4] += 1
        continue
    end
    if !residential[get(NYC_sales[i,:BUILDING_CLASS_CATEGORY])]
        removed[5] += 1
        continue
    end
    if NYC_sales[i,:BUILDING_CLASS_CATEGORY].isnull
        removed[6] += 1
        continue
    end
    if NYC_sales[i,:TAX_CLASS_AT_TIME_OF_SALE].isnull
        removed[7] += 1
        continue
    end
    if NYC_sales[i,:GROSS_SQUARE_FEET].isnull
        removed[8] += 1
        continue
    end
    if get(NYC_sales[i,:GROSS_SQUARE_FEET])<100.0
        removed[9] += 1
        continue
    end
    if get(NYC_sales[i,:logSalePricePerSQFT]) < 3.0
        removed[10] += 1
        # that's too cheap
        continue
    end
    if get(NYC_sales[i,:logSalePricePerSQFT]) > 8.0
        removed[11] += 1
        # that's too expensive
        continue
    end
    if NYC_sales[i,:XCoord].isnull
        removed[12] += 1
        continue
    end
    if NYC_sales[i,:YCoord].isnull
        removed[13] += 1
        continue
    end
    believable[i] = true
end

nobsv_ = sum(believable)
for_mm = copy(NYC_sales[believable,:])
reordering = collect(1:nobsv_)
schdistrs = sort(for_mm[:SchDistr].pool.levels)
istart = 1
for name in schdistrs
    schdistrs_col = for_mm[:SchDistr]
    indices = find(schdistrs_col.refs .== find(schdistrs_col.pool.index .== name)[1])
    nobsv_schdistr = length(indices)
    @printf("District %s has %d sales\n", name, nobsv_schdistr)
    iend = istart+nobsv_schdistr-1
    reordering[istart:iend,:] = indices
    istart = iend+1
end
for_mm = for_mm[reordering,:]
schdistr_indices = Dict{String,Vector{Int}}()
for name in schdistrs
    schdistrs_col = for_mm[:SchDistr]
    indices = find(schdistrs_col.refs .== find(schdistrs_col.pool.index .== name)[1])
    schdistr_indices[name] = indices
end
;
Y_dict=Dict{String, Vector{Float64}}()
X_dict=Dict{String, Array{Float64,2}}()
for name in schdistrs
    Y_dict[name] = for_mm[schdistr_indices[name], :logSalePricePerSQFT]
    X_dict[name] = for_mm[schdistr_indices[name],[:XCoord, :YCoord]]
end
sentinels_json=GeoJSON.parsefile("NYC_data/processed/SchoolDistrict_borders/SchoolDistrict_sentinels.json")
sentinels=Dict{Tuple{String,String},GeoJSON.MultiPoint}()
for f in sentinels_json.features
    key = (dec(f.properties["SchoolDistrict1"],2), dec(f.properties["SchoolDistrict2"],2))
    sentinels[key] = f.geometry
end

District 01 has 5 sales
District 02 has 85 sales
District 03 has 34 sales
District 04 has 6 sales
District 05 has 39 sales
District 06 has 30 sales
District 07 has 69 sales
District 08 has 739 sales
District 09 has 163 sales
District 10 has 312 sales
District 11 has 1172 sales
District 12 has 204 sales
District 13 has 322 sales
District 14 has 195 sales
District 15 has 517 sales
District 16 has 421 sales
District 17 has 459 sales
District 18 has 559 sales
District 19 has 686 sales
District 20 has 1010 sales
District 21 has 770 sales
District 22 has 1453 sales
District 23 has 245 sales
District 24 has 1283 sales
District 25 has 1317 sales
District 26 has 1268 sales
District 27 has 2199 sales
District 28 has 1033 sales
District 29 has 1956 sales
District 30 has 695 sales
District 31 has 0 sales
District 32 has 345 sales


In [46]:
X_dict["19"]

686×2 Array{Float64,2}:
 1.01245e6  186814.0
 1.01248e6  186595.0
 1.0125e6   186570.0
 1.01298e6  186757.0
 1.01209e6  185519.0
 1.01211e6  185521.0
 1.01331e6  185642.0
 1.01356e6  188194.0
 1.01323e6  187846.0
 1.01318e6  187612.0
 1.01437e6  187883.0
 1.0143e6   187837.0
 1.01539e6  187974.0
 ⋮                  
 1.0192e6   182341.0
 1.0194e6   182195.0
 1.0194e6   182195.0
 1.01931e6  182148.0
 1.0195e6   182350.0
 1.02361e6  182417.0
 1.02359e6  182476.0
 1.02357e6  182599.0
 1.02397e6  182390.0
 1.02393e6  182382.0
 1.02382e6  182673.0
 1.02125e6  180344.0

In [51]:
points_distr_dict = Dict{String,Vector{LibGEOS.Point}}()
for distr in keys(X_dict)
    X = X_dict[distr]
    points = [LibGEOS.Point(X[i,:]) for i in 1:size(X,1)]
    points_distr_dict[distr] = points
end

In [61]:
border = borders_dict[(19,27)]
distances_19 = [LibGEOS.distance(p, border) for p in points_distr_dict["19"]]
distances_27 = [LibGEOS.distance(p, border) for p in points_distr_dict["27"]]

2199-element Array{Float64,1}:
  6873.79 
  7380.36 
  6078.08 
 18277.2  
 17863.4  
 17581.2  
 16604.5  
 16394.1  
 17982.5  
 13329.9  
 13564.9  
 13694.1  
 13318.5  
     ⋮    
   758.551
  1602.21 
  2328.67 
  5467.29 
  5531.76 
   332.906
   123.798
   123.798
   345.143
   843.309
  2072.74 
  2285.45 

In [65]:
resid_multi = JSON.parsefile("NYC_data/processed/resid_multi.json")

Dict{String,Any} with 32 entries:
  "24" => Any[5.82215,5.27046,6.08294,5.83966,5.81488,5.7995,5.48396,5.44726,5.…
  "29" => Any[4.65263,5.41024,5.49985,5.56356,5.47666,5.65043,5.49956,5.77053,5…
  "32" => Any[6.4072,6.13528,6.83249,5.36659,6.4321,6.4974,6.28408,6.23172,5.13…
  "07" => Any[6.2389,4.6823,5.10428,4.63921,6.07306,4.77229,4.89776,4.40696,4.4…
  "12" => Any[4.56657,3.94776,5.84151,5.30726,4.15171,5.49041,4.18879,4.12291,5…
  "20" => Any[5.85571,6.34274,5.38755,5.76039,5.8591,3.01936,5.99472,5.58398,5.…
  "06" => Any[6.32931,6.395,6.19472,6.2479,6.71488,5.75223,6.04542,6.25457,6.26…
  "25" => Any[5.49132,5.96967,6.33895,6.22021,5.79593,5.98009,5.92144,5.82884,5…
  "03" => Any[6.30079,6.60451,6.86331,6.11695,6.57142,6.33068,6.64541,6.91508,6…
  "22" => Any[5.296,5.56968,5.17892,5.7061,5.55155,5.63543,5.58394,5.6944,5.638…
  "11" => Any[4.90563,3.61091,5.06685,5.0512,4.91019,5.1655,5.78627,5.25981,4.5…
  "23" => Any[4.95637,5.10909,4.78241,4.43588,4.93451,4.95898,4.84532,4.988

In [66]:
projected_df = DataFrame(distance    = [distances_19 ; -distances_27], 
                         Y           = [Y_dict["19"] ; Y_dict["27"]], 
                         Y_residuals = [resid_multi["19"] ; resid_multi["27"]])

,distance,Y,Y_residuals
1,8990.27,5.80747,5.46181
2,9000.46,5.69242,5.307
3,8981.09,6.84495,6.45953
4,8478.19,5.02638,4.64096
5,9586.55,5.37765,5.12684
6,9567.61,5.29365,5.04285
7,8366.19,5.40368,5.15287
8,7685.19,5.10373,4.75806
9,8079.09,5.85757,5.34634
10,8162.61,4.86222,4.51655


In [67]:
DataFrames.writetable("NYC_data/processed/projected_19_27.csv", projected_df)